# Random Forest Regression


### 1. 변수 정리 , 2. 가지수 변경

In [1]:
# RMSE 함수
from sklearn.metrics import make_scorer

def rmse(predicted_values, actual_values):
    # 넘파이로 배열 형태로 바꿔준다.
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)
        
    # 위에서 계산한 예측값에서 실제값을 빼주고 제곱을 해준다.
    difference = predicted_values - actual_values
    # difference = (log_predict - log_actual) ** 2
    difference = np.square(difference)
    
    # 평균을 낸다.
    mean_difference = difference.mean()
    
    # 다시 루트를 씌운다.
    score = np.sqrt(mean_difference)
    
    return score

In [4]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
train = pd.read_csv('./duranduranreturns_data/20190119_제출용총train.csv', encoding='utf-8')
train.head()

,apartment_id,Seoul,Busan,transaction_date,year_of_completion,latitude,longitude,central,district,individual,...,supply_area,tallest_building_in_sites,lowest_building_in_sites,total_household_count_in_sites,total_household_count_of_area_type,total_parking_capacity_in_site,household_loan,mean_loan,mortgage loan,transaction_real_price
0,5584,0,1,20,1999,37.585965,127.000231,0,0,1,...,0.112894,0.040000,0.043478,0.005469,0.015541,0.016691,0.568465,0.561404,0.605691,215000000.0
1,5584,0,1,20,1999,37.585965,127.000231,0,0,1,...,0.102380,0.040000,0.043478,0.005469,0.003378,0.016691,0.568465,0.561404,0.605691,200000000.0
2,5059,0,1,20,1992,37.580511,127.014016,0,0,1,...,0.129581,0.120000,0.115942,0.071840,0.067905,0.092361,0.568465,0.561404,0.605691,168000000.0
3,2816,0,1,20,1993,37.580324,127.011788,0,0,1,...,0.166625,0.133333,0.144928,0.121212,0.095946,0.092361,0.568465,0.561404,0.605691,165000000.0
4,2816,0,1,20,1993,37.580324,127.011788,0,0,1,...,0.266898,0.133333,0.144928,0.121212,0.037838,0.092361,0.568465,0.561404,0.605691,280000000.0


In [6]:
train2 = pd.read_csv('./duranduranreturns_data/nonull_train.csv', encoding='utf-8')
train2.head()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,key,apartment_id,city,year_of_completion,exclusive_use_area,floor,latitude,longitude,address_by_law,...,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price,household_loan,mean_loan,mortgage_loan,transaction_year_month_date
0,0,0,5584,1,1999,47.43,6,37.585965,127.000231,1111017100,...,65.63,46,1.0,1.0,corridor,215000000,5.69,5.79,5.64,2006-01-20
1,1,1,5584,1,1999,44.37,8,37.585965,127.000231,1111017100,...,61.39,10,2.0,1.0,corridor,200000000,5.69,5.79,5.64,2006-01-20
2,2,2,5059,1,1992,54.70,8,37.580511,127.014016,1111017400,...,72.36,201,2.0,1.0,corridor,168000000,5.69,5.79,5.64,2006-01-20
3,3,3,2816,1,1993,64.66,11,37.580324,127.011788,1111017400,...,87.30,284,2.0,1.0,corridor,165000000,5.69,5.79,5.64,2006-01-20
4,4,4,2816,1,1993,106.62,7,37.580324,127.011788,1111017400,...,127.74,112,4.0,2.0,stairway,280000000,5.69,5.79,5.64,2006-01-20


In [7]:
train['key'] = train2['key']

In [8]:
train.columns

Index(['apartment_id', 'Seoul', 'Busan', 'transaction_date',
       'year_of_completion', 'latitude', 'longitude', 'central', 'district',
       'individual', 'cogeneration', 'gas', 'corridor', 'mixed', 'stairway',
       'city1', 'city2', 'city3', 'city4', 'transaction_year',
       'transaction_month', 'apartment_building_count_in_sites',
       'bathroom_count', 'exclusive_use_area', 'floor', 'room_count',
       'room_id', 'supply_area', 'tallest_building_in_sites',
       'lowest_building_in_sites', 'total_household_count_in_sites',
       'total_household_count_of_area_type', 'total_parking_capacity_in_site',
       'household_loan', 'mean_loan', 'mortgage loan',
       'transaction_real_price', 'key'],
      dtype='object')

In [9]:
train = train[['key', 'apartment_id', 'Seoul', 'Busan', 'transaction_date',
       'year_of_completion', 'central', 'district',
       'individual', 'cogeneration', 'gas', 'corridor', 'mixed', 'stairway',
       'transaction_year',
       'transaction_month', 'apartment_building_count_in_sites',
       'bathroom_count', 'floor', 'room_count',
       'room_id', 'supply_area', 'tallest_building_in_sites',
       'lowest_building_in_sites',
       'total_household_count_of_area_type',
       'household_loan', 'mean_loan', 'mortgage loan',
       'transaction_real_price']]

In [10]:
train.head()

,key,apartment_id,Seoul,Busan,transaction_date,year_of_completion,central,district,individual,cogeneration,...,room_count,room_id,supply_area,tallest_building_in_sites,lowest_building_in_sites,total_household_count_of_area_type,household_loan,mean_loan,mortgage loan,transaction_real_price
0,0,5584,0,1,20,1999,0,0,1,0,...,0.125,0.473926,0.112894,0.040000,0.043478,0.015541,0.568465,0.561404,0.605691,215000000.0
1,1,5584,0,1,20,1999,0,0,1,0,...,0.250,0.473920,0.102380,0.040000,0.043478,0.003378,0.568465,0.561404,0.605691,200000000.0
2,2,5059,0,1,20,1992,0,0,1,0,...,0.250,0.043784,0.129581,0.120000,0.115942,0.067905,0.568465,0.561404,0.605691,168000000.0
3,3,2816,0,1,20,1993,0,0,1,0,...,0.250,0.030306,0.166625,0.133333,0.144928,0.095946,0.568465,0.561404,0.605691,165000000.0
4,4,2816,0,1,20,1993,0,0,1,0,...,0.500,0.030290,0.266898,0.133333,0.144928,0.037838,0.568465,0.561404,0.605691,280000000.0


In [11]:
# dataset.columns, 
len(train.columns)
# dataset.iloc[:,0:2]

29

In [13]:
# Data Division
X = train.iloc[:, 1:28].values
y = train.iloc[:, 28].values

In [14]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [15]:
X_train

array([[2.95500000e+03, 0.00000000e+00, 1.00000000e+00, ...,
        5.45643154e-01, 5.72368421e-01, 4.69512195e-01],
       [4.09000000e+03, 0.00000000e+00, 1.00000000e+00, ...,
        1.24481328e-01, 1.42543860e-01, 1.30081301e-01],
       [7.73900000e+03, 1.00000000e+00, 0.00000000e+00, ...,
        4.56431535e-01, 4.58333333e-01, 3.98373984e-01],
       ...,
       [4.06800000e+03, 0.00000000e+00, 1.00000000e+00, ...,
        5.95435685e-01, 6.16228070e-01, 6.15853659e-01],
       [2.31230000e+04, 0.00000000e+00, 1.00000000e+00, ...,
        1.57676349e-01, 1.00877193e-01, 1.64634146e-01],
       [1.04380000e+04, 1.00000000e+00, 0.00000000e+00, ...,
        8.65145228e-01, 8.53070175e-01, 8.65853659e-01]])

In [16]:
X_test

array([[2.81500000e+03, 0.00000000e+00, 1.00000000e+00, ...,
        1.47302905e-01, 8.77192982e-02, 1.42276423e-01],
       [3.81200000e+03, 0.00000000e+00, 1.00000000e+00, ...,
        5.66390041e-01, 5.83333333e-01, 5.69105691e-01],
       [7.41000000e+02, 0.00000000e+00, 1.00000000e+00, ...,
        4.35684647e-02, 4.82456140e-02, 4.67479675e-02],
       ...,
       [5.62000000e+02, 1.00000000e+00, 0.00000000e+00, ...,
        2.26141079e-01, 2.60964912e-01, 2.09349593e-01],
       [4.36400000e+03, 0.00000000e+00, 1.00000000e+00, ...,
        6.39004149e-01, 6.42543860e-01, 6.50406504e-01],
       [9.24300000e+03, 1.00000000e+00, 0.00000000e+00, ...,
        2.28215768e-01, 2.67543860e-01, 2.13414634e-01]])

In [17]:
# Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 50, random_state = 0)
regressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

## 분할 train 결과 비교

In [18]:
# Predicting a new result
y_train_pred = regressor.predict(X_train)

In [19]:
rmse(y_train_pred, y_train)

13889460.757406434

## 분할 test 결과 비교

In [20]:
y_test_pred = regressor.predict(X_test)

In [21]:
rmse(y_test_pred, y_test)

35716413.59736318

## 전체 train 결과 비교

In [38]:
y_dataset = regressor.predict(X)

In [39]:
rmse(y_dataset, y)

21075578.29877149

# test 데이터 불러오기

In [22]:
test = pd.read_csv('./duranduranreturns_data/20190119_제출용총test.csv', encoding='utf-8')

In [24]:
test2 = pd.read_csv('./duranduranreturns_data/nonull_test.csv', encoding='utf-8')
test2.head()

,Unnamed: 0,key,apartment_id,city,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,...,supply_area,total_household_count_of_area_type,room_count,bathroom_count,front_door_structure,transaction_real_price,household_loan,mean_loan,mortgage_loan,transaction_year_month_date
0,0,462533,3751,1,200912,31,1984,83.58,14,37.519926,...,107.19,2,3.0,1.0,corridor,0,5.95,5.81,5.90,2009-12-31
1,1,764018,14029,1,201304,10,1968,107.17,10,37.572215,...,107.17,10,0.0,0.0,corridor,0,4.42,4.73,3.86,2013-04-10
2,2,813528,12304,1,201309,30,1968,36.17,6,37.570067,...,39.66,58,2.0,1.0,stairway,0,4.26,4.54,3.82,2013-09-30
3,3,845097,22241,1,201312,10,2007,45.67,8,37.556938,...,87.11,1,3.0,2.0,stairway,0,4.10,4.52,3.74,2013-12-10
4,4,856338,316,1,201401,10,1988,41.85,4,37.652256,...,56.70,90,2.0,1.0,stairway,0,4.15,4.53,3.75,2014-01-10


In [25]:
test['key'] = test2['key']

In [29]:
len(test.columns)

29

In [27]:
test = test[['key', 'apartment_id', 'Seoul', 'Busan', 'transaction_date',
       'year_of_completion', 'central', 'district',
       'individual', 'cogeneration', 'gas', 'corridor', 'mixed', 'stairway',
       'transaction_year',
       'transaction_month', 'apartment_building_count_in_sites',
       'bathroom_count', 'floor', 'room_count',
       'room_id', 'supply_area', 'tallest_building_in_sites',
       'lowest_building_in_sites',
       'total_household_count_of_area_type',
       'household_loan', 'mean_loan', 'mortgage loan',
       'transaction_real_price']]

In [30]:
# Data Division
X2 = test.iloc[:, 1:28].values
y2 = test.iloc[:, 28].values

# Test 데이터로부터 예측값 추출하기 

In [31]:
prediction = regressor.predict(X2)

In [32]:
prediction

array([7.42390000e+08, 3.68740000e+08, 1.27800000e+08, ...,
       7.00280000e+08, 7.21288333e+08, 1.92890000e+08])

# 결과값 저장하기

In [33]:
submission = pd.read_csv("./duranduranreturns_data/data/submission.csv")
submission

submission["transaction_real_price"] = prediction

print(submission.shape)
submission.head()

(3918, 2)


,key,transaction_real_price
0,462533,742390000.0
1,764018,368740000.0
2,813528,127800000.0
3,845097,409214200.0
4,856338,166946000.0


In [34]:
submission.to_csv("./duranduranreturns_data/submission/20190122_submission_rf3_ysw.csv",index=False, encoding= 'utf-8')

## (참고) 결과값 다른 모델과 비교해보기

In [40]:
submission2 = pd.read_csv('./duranduranreturns_data/submission/20190122_submission_rf1_ysw.csv', encoding='utf-8')
submission2.head()

,key,transaction_real_price
0,462533,838300000.0
1,764018,377075000.0
2,813528,125662500.0
3,845097,237175000.0
4,856338,168575000.0


In [42]:
submission3 = pd.read_csv('./duranduranreturns_data/submission/20190122_submission_rf2_ysw.csv', encoding='utf-8')
submission3.head()

,key,transaction_real_price
0,462533,792395500.0
1,764018,374987500.0
2,813528,140312500.0
3,845097,240407500.0
4,856338,153956250.0


In [43]:
rmse(submission['transaction_real_price'], submission2['transaction_real_price']), rmse(submission['transaction_real_price'], submission3['transaction_real_price'])

(81971976.66624208, 86647735.81755851)

In [37]:
rmse(submission2['transaction_real_price'], submission3['transaction_real_price'])

9620136.085834995